In [ ]:
# -----------------------------------------------------------------------------------
# 파일명       : LSTM_model.ipynb
# 설명         : LSTM 모델 학습
# 작성자       : 이민하
# 작성일       : 2024-11-13
# 
# 사용 모듈    :
# - pandas                           # 데이터프레임 기반 데이터 처리
# - pickle                           # 객체 저장 및 로딩 (직렬화)
# - os                               # 파일 및 경로 관리
# - torch, torch.nn, F               # PyTorch 모델 구축 및 연산
# - torch.utils.data                 # 데이터셋 및 데이터로더 처리
# - sklearn.model_selection          # 학습/검증용 데이터 분할
# - sklearn.preprocessing            # 데이터 정규화 및 스케일링
# - torch.optim, lr_scheduler        # 최적화 및 학습률 조정
# - torchmetrics.regression          # 회귀 모델 평가 지표 계산
# -----------------------------------------------------------------------------------
# >> 주요 기능
# - 데이터 및 모델 불러오기
# - 모델 학습
#
# >> 업데이트 내역
# [2024-11-13] 이상 패턴 제거 전 데이터 학습
# [2024-11-14] 데이터 변경 (56 dimensions)
# [2024-11-28] 데이터 변경 (이상 패턴 제거 후 29 dimensions)
# -----------------------------------------------------------------------------------


In [ ]:
# 데이터프레임 기반 데이터 처리
import pandas as pd

# 객체 저장 및 로딩 (직렬화)
import pickle

# 경로 관리
import os

# PyTorch 모델 구축 및 연산
import torch
import torch.nn as nn
import torch.nn.functional as F

# 데이터셋 및 데이터로더 처리
from torch.utils.data import Dataset, DataLoader

# 학습/검증용 데이터 분할
from sklearn.model_selection import train_test_split

# 데이터 정규화 및 스케일링
from sklearn.preprocessing import MinMaxScaler, RobustScaler

# 최적화 및 학습률 조정
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

# 회귀 모델 평가 지표 계산
from torchmetrics.regression import R2Score, MeanAbsoluteError, MeanAbsolutePercentageError, MeanSquaredError

In [ ]:
# 데이터 경로 설정
DATA_PATH = '../Data/'

# 29 dimensions를 가진 데이터 불러오기 (이상 패턴 제거)
electric_df = pd.read_csv(DATA_PATH + 'electric_df_clear_29_days.csv')
water_df = pd.read_csv(DATA_PATH + 'water_df_clear_29_days.csv')

In [3]:
# electric_df

In [4]:
water_df

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,30,120,210,410,32,184,180,260,35,145,...,95,46,139,204,198,53,162,210,150,51
1,120,210,410,32,184,180,260,35,145,203,...,46,139,204,198,53,162,210,150,51,169
2,210,410,32,184,180,260,35,145,203,216,...,139,204,198,53,162,210,150,51,169,204
3,410,32,184,180,260,35,145,203,216,43,...,204,198,53,162,210,150,51,169,204,169
4,32,184,180,260,35,145,203,216,43,136,...,198,53,162,210,150,51,169,204,169,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285993,157,230,169,47,122,218,350,110,183,190,...,58,218,290,105,10,68,38,263,78,158
285994,230,169,47,122,218,350,110,183,190,230,...,218,290,105,10,68,38,263,78,158,290
285995,169,47,122,218,350,110,183,190,230,98,...,290,105,10,68,38,263,78,158,290,300
285996,47,122,218,350,110,183,190,230,98,143,...,105,10,68,38,263,78,158,290,300,280


In [ ]:
# Train, Test 데이터 나누기 (Test Size = 0.2)

# electric_features = electric_df[electric_df.columns[:-1]]
# electric_target = electric_df[electric_df.columns[-1:]]

# electric_X_train, electric_X_test, electric_y_train, electric_y_test = train_test_split(electric_features,
#                                                     electric_target,
#                                                     random_state = 42,
                                                    # test_size = 0.2)

water_features = water_df[water_df.columns[:-1]]
water_target = water_df[water_df.columns[-1:]]

water_X_train, water_X_test, water_y_train, water_y_test = train_test_split(water_features,
                                                                            water_target,
                                                                            random_state = 42,
                                                                            test_size = 0.2)

In [6]:
# electric_X_test

In [7]:
# electric_y_train.info()

In [ ]:
# 모델 연산을 위한 데이터셋
class CustomDataset(Dataset):
    def __init__(self, featureDF, targetDF):
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = self.featureDF.shape[0]
        self.n_cols = self.featureDF.shape[1]

    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)

        return featureTS, targetTS

In [ ]:
# LSTM 모델
class LSTMModel(nn.Module):
    def __init__(self, hidden_dim, input_size, n_layers, dropout,
                 bidirectional):
        super().__init__()

        # LSTM 모델
        self.model = nn.LSTM(
            input_size = input_size,
            hidden_size = hidden_dim,
            num_layers = n_layers,
            dropout = dropout,
            bidirectional = bidirectional,
            batch_first = True
        )

        # 출력층 
        # 양방향 LSTM (시퀀스 데이터에서 더 많은 정보 추출 가능)
        if bidirectional:
            self.linear = nn.Linear(hidden_dim * 2, 1)
        
        else:
            self.linear = nn.Linear(hidden_dim, 1)

        # 성능에 따라 추가
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        output, _ = self.model(inputs)
        logits = self.linear(output)

        return logits

In [ ]:
# 전기 데이터는 MinMaxScaler를 통해 스케일링
# electric_mmscaler = MinMaxScaler().fit(electric_X_train)

# 수도 데이터는 RobustScaler를 통해 스케일링
water_rbscaler = RobustScaler().fit(water_X_train)

# pickle 모듈을 통해 전기 스케일러 저장
# with open('electric_min_max_scaler.pkl', 'wb') as f:
#     pickle.dump(electric_mmscaler, f)

# pickle 모듈을 통해 수도 스케일러 저장
with open('water_robust_scaler.pkl', 'wb') as f:
    pickle.dump(water_rbscaler, f)

In [ ]:
# 전기, 수도 데이터를 스케일링된 데이터로 변경
# electric_X_train_scaled = electric_mmscaler.transform(electric_X_train)
# electric_X_test_scaled = electric_mmscaler.transform(electric_X_test)

water_X_train_scaled = water_rbscaler.transform(water_X_train)
water_X_test_scaled = water_rbscaler.transform(water_X_test)

# 스케일링된 데이터로 데이터프레임 재구성
# electric_X_train = pd.DataFrame(electric_X_train_scaled, columns = electric_X_train.columns)
# electric_X_test = pd.DataFrame(electric_X_test_scaled, columns = electric_X_test.columns)

water_X_train = pd.DataFrame(water_X_train_scaled, columns = water_X_train.columns)
water_X_test = pd.DataFrame(water_X_test_scaled, columns = water_X_test.columns)

In [ ]:
# 학습 파라미터 설정
EPOCH = 1000
BATCH_SIZE = 64
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

In [ ]:
# 데이터셋, 데이터로더 변환
# electric_trainDS = CustomDataset(electric_X_train, electric_y_train)
water_trainDS = CustomDataset(water_X_train, water_y_train)

# electric_trainDL = DataLoader(electric_trainDS, batch_size = BATCH_SIZE)
water_trainDL = DataLoader(water_trainDS, batch_size = BATCH_SIZE)

In [ ]:
# 모델 파라미터 설정
input_size = 28
hidden_dim = 32
n_layers = 2
dropout = 0.5

# 모델 생성
lstm_model = LSTMModel(input_size = input_size, hidden_dim = hidden_dim,
                       n_layers = n_layers, dropout = 0.8, bidirectional = True).to(DEVICE)


In [ ]:
# 손실 함수 생성
MAEloss = MeanAbsoluteError()
MAPEloss = MeanAbsolutePercentageError()
MSEloss = MeanSquaredError()
R2score = R2Score()

# 옵티마이저 생성
optimizer = optim.RMSprop(lstm_model.parameters(), lr = LR)

# Learning Rate Scheduler 생성
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', patience = 10, verbose = True)

c:\Users\KDP-2\anaconda3\envs\Project_38\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
# 모델 Test 함수
def testing(featureDF, targetDF, model):
    # Pytorch 학습을 위해 데이터프레임 -> 텐서 전환
    featureTS = torch.FloatTensor(featureDF.values).to(DEVICE)
    targetTS = torch.FloatTensor(targetDF.values).to(DEVICE)

    # Dropout, BatchNorm 등 가중치 규제 비활성화    
    model.eval()
    
    # 평가를 위해 역전파 계산 X
    with torch.no_grad():
        pre_val = model(featureTS)
        mae_loss_val = MAEloss(pre_val, targetTS)
        mape_loss_val = MAPEloss(pre_val, targetTS)
        mse_loss_val = MSEloss(pre_val, targetTS)
        score_val = R2score(pre_val, targetTS)
    
    return mae_loss_val, mape_loss_val, mse_loss_val, score_val, pre_val

In [ ]:
# 모델 Train 함수
def training(testDF, testtargetDF, model, trainDL, test_value):
    
    # 가중치 파일 저장 위치 정의
    SAVE_PATH = './saved_models/'
    os.makedirs(SAVE_PATH, exist_ok = True)
    
    # Early Stopping을 위한 변수
    BREAK_CNT_LOSS = 0
    BREAK_CNT_SCORE = 0
    LIMIT_VALUE = 10

    # Loss가 더 낮은 가중치 파일을 저장하기 위하여 Loss 로그를 담을 리스트
    MAE_LOSS_HISTORY, MAPE_LOSS_HISTORY, MSE_LOSS_HISTORY, SCORE_HISTORY = [[], []], [[], []], [[], []], [[], []]

    for epoch in range(1, EPOCH + 1):
        SAVE_MODEL = os.path.join(SAVE_PATH, f'model_{epoch}.pth')
        SAVE_WEIGHT = os.path.join(SAVE_PATH, f'model_weights_{epoch}.pth')

        mae_loss_total, mape_loss_total, mse_loss_total, score_total = 0, 0, 0, 0

        # Train DataLoader에 저장된 feature, target 텐서로 학습 진행
        for featureTS, targetTS in trainDL:
            # 결과 추론
            pre_y = model(featureTS)
            
            # 추론값으로 Loss값 계산
            mae_loss = MAEloss(pre_y, targetTS)
            mape_loss = MAPEloss(pre_y, targetTS)
            mse_loss = MSEloss(pre_y, targetTS)

            mae_loss_total += mae_loss.item()
            mape_loss_total += mape_loss.item()
            mse_loss_total += mse_loss.item()

            score = R2score(pre_y, targetTS)
            score_total += score.item()

            # mae, mape, mse loss 전부 더한 값으로 기울기 
            total_loss = mae_loss + mape_loss + mse_loss

            # 이전 gradient 초기화
            optimizer.zero_grad()

            # 역전파로 gradient 계산
            total_loss.backward()

            # 계산된 gradient로 가중치 업데이트
            optimizer.step()

        # test loss, score, 예측값도 계산
        test_mae_loss, test_mape_loss, test_mse_loss, test_score, pre_val = testing(testDF, testtargetDF, model)

        MAE_LOSS_HISTORY[1].append(test_mae_loss)
        MAPE_LOSS_HISTORY[1].append(test_mape_loss)
        MSE_LOSS_HISTORY[1].append(test_mse_loss)
        SCORE_HISTORY[1].append(test_score)

        MAE_LOSS_HISTORY[0].append(mae_loss_total / len(trainDL))
        MAPE_LOSS_HISTORY[0].append(mape_loss_total / len(trainDL))
        MSE_LOSS_HISTORY[0].append(mse_loss_total / len(trainDL))
        SCORE_HISTORY[0].append(score_total / len(trainDL))

        print(f'pre_val : {pre_val.squeeze().tolist()[:10]}\ny_val : {test_value.values.squeeze()[:10]}\n')
        print(f'[{epoch} / {EPOCH}]\n- TRAIN MAE LOSS : {MAE_LOSS_HISTORY[0][-1]}')
        print(f'- TRAIN MAPE LOSS : {MAPE_LOSS_HISTORY[0][-1]}')
        print(f'- TRAIN MSE LOSS : {MSE_LOSS_HISTORY[0][-1]}')
        print(f'- TRAIN R2 SCORE : {SCORE_HISTORY[0][-1]}')

        print(f'\n- TEST MAE LOSS : {MAE_LOSS_HISTORY[1][-1]}')
        print(f'- TEST MAPE LOSS : {MAPE_LOSS_HISTORY[1][-1]}')
        print(f'- TEST MSE LOSS : {MSE_LOSS_HISTORY[1][-1]}')
        print(f'- TEST R2 SCORE : {SCORE_HISTORY[1][-1]}')

        # test loss 결과로 스케줄러 업데이트
        scheduler.step(test_mae_loss)

        # Early Stopping 구현
        if len(MAE_LOSS_HISTORY[1]) >= 2:
            if MAE_LOSS_HISTORY[1][-1] >= MAE_LOSS_HISTORY[1][-2]: BREAK_CNT_LOSS += 1
        
        if len(MAE_LOSS_HISTORY[1]) == 1:
            torch.save(model.state_dict(), SAVE_WEIGHT)
            torch.save(model, SAVE_MODEL)

        else:
            if MAE_LOSS_HISTORY[1][-1] < min(MAE_LOSS_HISTORY[1][:-1]):
                torch.save(model.state_dict(), SAVE_WEIGHT)
                torch.save(model, SAVE_MODEL)

        if BREAK_CNT_LOSS > LIMIT_VALUE:
            print(f"성능 및 손실 개선이 없어서 {epoch} EPOCH에 학습 중단")
            break

    return MAE_LOSS_HISTORY, MAPE_LOSS_HISTORY, MSE_LOSS_HISTORY, SCORE_HISTORY
        

In [ ]:
# 모델 학습 시작
mae_loss, mape_loss, mse_loss, r2 = training(water_X_test, water_y_test, lstm_model, water_trainDL, water_y_test)


NameError: name 'training' is not defined